In [1]:
import pandas as pd
import numpy as np
import glob
import os
import gspread
from gspread_dataframe import get_as_dataframe, set_with_dataframe
from oauth2client.service_account import ServiceAccountCredentials

folder_path = r'G:\Fund_analysis\Raw_data'
csv_files = glob.glob(os.path.join(folder_path, '*.csv'))

def process_files(files, identifier):
    dataframes_list = []
    
    for file in files:
        df = pd.read_csv(file)

        if 'Debt' in file:
            df['Fund Type'] = 'Debt'
        elif 'Equity' in file:
            df['Fund Type'] = 'Equity'
        elif 'Hybrid' in file:
            df['Fund Type'] = 'Hybrid'
        elif 'Commodity' in file:
            df['Fund Type'] = 'Commodity'
        else:
            pass

        print(f"Processed file: {file}")
        dataframes_list.append(df)

    if dataframes_list:
        combined_df = pd.concat(dataframes_list, ignore_index=True)
        print(f"Combined DataFrame {identifier}: Length = {len(combined_df)}")
        return combined_df
    else:
        print(f"No files with '{identifier}' in the name were found.")
        return None

def add_deviation_volatility(data):
    data['Diff Volatility Caty'] = data['Volatility'] - data['Category St Dev']
    print("Added 'Diff Volatility Cat' Columns............")

    return data

files_with_one = [file for file in csv_files if '1' in file]
files_with_two = [file for file in csv_files if '2' in file]
files_with_three = [file for file in csv_files if '3' in file]

df_1 = process_files(files_with_one, '1')
df_2 = process_files(files_with_two, '2')
df_3 = process_files(files_with_three, '3')

if df_1 is not None and df_2 is not None and df_3 is not None:
    combined_df = pd.merge(df_1, df_2, on=df_1.columns[0], how='outer')
    combined_df = pd.merge(combined_df, df_3, on=combined_df.columns[0], how='outer')

    # Drop columns with '_x' and '_y' suffixes, but keep the original column
    columns_to_drop = [col for col in combined_df.columns if col.endswith('_x') or col.endswith('_y')]
    combined_df.drop(columns=columns_to_drop, inplace=True)

    # Exclude any columns that contain '%Other%' in their name
    columns_to_keep = [col for col in combined_df.columns if 'Other' not in col]
    combined_df = combined_df[columns_to_keep]

   # Replace 0 with NaN
    combined_df.replace(0, np.nan, inplace=True)

    # Format float columns to 3 decimal places, except 'Expense Ratio'
    float_cols = combined_df.select_dtypes(include='float64').columns
    for col in float_cols:
        if col != 'Expense Ratio':
            combined_df[col] = combined_df[col].round(3)

combined_df = add_deviation_volatility(combined_df)
print("Dataframe combined")

Processed file: G:\Fund_analysis\Raw_data\MF_Commodity_1.csv
Processed file: G:\Fund_analysis\Raw_data\MF_Debt_1.csv
Processed file: G:\Fund_analysis\Raw_data\MF_Equity_1.csv
Processed file: G:\Fund_analysis\Raw_data\MF_Hybrid_1.csv
Combined DataFrame 1: Length = 1461
Processed file: G:\Fund_analysis\Raw_data\MF_Commodity_2.csv
Processed file: G:\Fund_analysis\Raw_data\MF_Debt_2.csv
Processed file: G:\Fund_analysis\Raw_data\MF_Equity_2.csv
Processed file: G:\Fund_analysis\Raw_data\MF_Hybrid_2.csv
Combined DataFrame 2: Length = 1461
Processed file: G:\Fund_analysis\Raw_data\MF_Commodity_3.csv
Processed file: G:\Fund_analysis\Raw_data\MF_Debt_3.csv
Processed file: G:\Fund_analysis\Raw_data\MF_Equity_3.csv
Processed file: G:\Fund_analysis\Raw_data\MF_Hybrid_3.csv
Combined DataFrame 3: Length = 1461
Added 'Diff Volatility Cat' Columns............


In [2]:

# Define the scope for Google Sheets and Drive APIs
scope = ['https://www.googleapis.com/auth/spreadsheets', 'https://www.googleapis.com/auth/drive']
cred_path = r'G:\Fund_analysis_cred\cred_file.json'

creds = ServiceAccountCredentials.from_json_keyfile_name(cred_path, scope)
client = gspread.authorize(creds)

spreadsheet_id = '15Tx7fwzYAQLlXEz8nr3zcSuIWT1gn0GnhjmzCnpW5PE'
spreadsheet = client.open_by_key(spreadsheet_id) 

worksheet = spreadsheet.get_worksheet(0)
worksheet.clear()

set_with_dataframe(worksheet, combined_df)

In [3]:
combined_df.columns

Index(['Name', 'Expense Ratio', 'Absolute Returns - 3M',
       'Absolute Returns - 6M', 'Absolute Returns - 1Y', 'CAGR 3Y', 'CAGR 5Y',
       'CAGR 10Y', 'Alpha', 'Volatility', 'Category St Dev',
       'SEBI Risk Category', '% Debt Holding', '% Equity Holding',
       '% Largecap Holding', '% Midcap Holding', '% Smallcap Holding',
       'PE Ratio', 'Category PE Ratio', 'Sharpe Ratio', 'Sortino Ratio',
       'Sub Category', 'Plan', 'AUM', 'Time since inception', 'Benchmark',
       'Exit Load', 'Lock-in', 'Minimum Lumpsum', 'Minimum SIP', 'NAV',
       'Fund Type', 'Diff Volatility Caty'],
      dtype='object')